<h1 style="text-align: center;"> TP 02</h1>
<h2> OUSLIMANI Hocine</br> ABOURA Mohammed Ilyes</h>

In [24]:
from Bio import Entrez, SeqIO
from Bio.Seq import Seq
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio import AlignIO
import os
import pandas as pd
import random
import numpy as np
from tabulate import tabulate
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [25]:
# Configuration des paramètres
Entrez.email = "getBioPythonSequence@example.com"
types = ["alpha", "beta", "omega", "delta"]
years = [2020, 2021, 2022, 2023, 2024]
num_sequences_to_save = 1  # Nombre de séquences les plus courtes à sauvegarder
sequences_to_fetch = 50  # Nombre de séquences à examiner par recherche

# Longueur minimale des séquences
min_length = 300

# Dossier principal pour stocker les séquences
output_dir = "SARS-CoV-2_Sequences"

In [ ]:
# Fonction pour créer un dossier s'il n'existe pas
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Fonction pour convertir une séquence ADN en ARN
def convert_dna_to_rna(sequence):
    return sequence.replace("T", "U")

# Fonction pour trouver les séquences les plus courtes
def find_shortest_sequences(query):
    print(f"Recherche de {sequences_to_fetch} séquences pour : {query}")
    try:
        # Recherche des séquences dans NCBI
        handle = Entrez.esearch(db="nucleotide", term=query, retmax=sequences_to_fetch)
        record = Entrez.read(handle)
        handle.close()

        id_list = record["IdList"]
        if not id_list:
            print("Aucune séquence trouvée.")
            return []

        # Récupération des séquences
        handle = Entrez.efetch(db="nucleotide", id=id_list, rettype="fasta", retmode="text")
        fasta_records = list(SeqIO.parse(handle, "fasta"))
        handle.close()

        # Affichage des séquences récupérées avec leur taille
        print("\nSéquences récupérées :")
        for record in fasta_records:
            print(f"- ID : {record.id}, Taille : {len(record.seq)} bases")

        # Filtrer les séquences valides (au-dessus de la longueur minimale)
        valid_sequences = [
            record for record in fasta_records if len(record.seq) >= min_length
        ]

        if not valid_sequences:
            print("Aucune séquence valide trouvée.")
            return []

        # Trier par longueur croissante
        valid_sequences.sort(key=lambda record: len(record.seq))

        # Retourner les plus courtes
        return valid_sequences[:num_sequences_to_save]

    except Exception as e:
        print(f"Erreur lors de la récupération des séquences : {e}")
        return []

# Fonction pour télécharger et sauvegarder les séquences depuis NCBI
def download_fasta_sequences():
    for sars_type in types:
        for year in years:
            query = f"SARS-CoV-2 {sars_type} {year}[pdat]"
            print(f"\nRecherche : {query}")

            # Créer le dossier correspondant
            folder_path = os.path.join(output_dir, sars_type, str(year))
            create_folder(folder_path)

            # Trouver les séquences les plus courtes
            shortest_sequences = find_shortest_sequences(query)
            if not shortest_sequences:
                print("Aucune séquence à sauvegarder. Recherche abandonnée pour ce type et cette année.")
                continue

            # Sauvegarder les séquences
            for record in shortest_sequences:
                # Convertir la séquence en ARN
                rna_sequence = convert_dna_to_rna(str(record.seq))

                # Modifier le record pour inclure la séquence ARN
                record.seq = Seq(rna_sequence)

                # Sauvegarder la séquence
                fasta_file = os.path.join(folder_path, f"{record.id}.fasta")
                SeqIO.write(record, fasta_file, "fasta")
                print(f"Séquence ARN sauvegardée : {fasta_file}")

# Lancer le téléchargement
download_fasta_sequences()



Recherche : SARS-CoV-2 alpha 2020[pdat]
Recherche de 50 séquences pour : SARS-CoV-2 alpha 2020[pdat]

Séquences récupérées :
- ID : NM_001377530.1, Taille : 7984 bases
- ID : NM_001388412.1, Taille : 6256 bases
- ID : NM_001388417.1, Taille : 6758 bases
- ID : NM_001388415.1, Taille : 4594 bases
- ID : NM_001388414.1, Taille : 4606 bases
- ID : NM_001388418.1, Taille : 6731 bases
- ID : NM_001388416.1, Taille : 4567 bases
- ID : NM_001388419.1, Taille : 16188 bases
- ID : NM_001388413.1, Taille : 4633 bases
- ID : NM_001379692.1, Taille : 3996 bases
- ID : NR_170953.1, Taille : 4693 bases
- ID : NR_170952.1, Taille : 4654 bases
- ID : MW027019.1, Taille : 313 bases
Séquence ARN sauvegardée : SARS-CoV-2_Sequences\alpha\2020\MW027019.1.fasta

Recherche : SARS-CoV-2 alpha 2021[pdat]
Recherche de 50 séquences pour : SARS-CoV-2 alpha 2021[pdat]

Séquences récupérées :
- ID : NM_001394000.1, Taille : 1663 bases
- ID : NM_001393999.1, Taille : 1637 bases
- ID : OL689583.1, Taille : 29855 bas

In [26]:
# Fonction pour calculer le pourcentage de chaque base
def calculate_base_percentages(sequence):
    total_bases = len(sequence)
    if total_bases == 0:
        return {"A": 0, "U": 0, "C": 0, "G": 0}

    percentages = {
        "A": (sequence.count("A") / total_bases) * 100,
        "U": (sequence.count("U") / total_bases) * 100,
        "C": (sequence.count("C") / total_bases) * 100,
        "G": (sequence.count("G") / total_bases) * 100,
    }
    return percentages

# Fonction pour afficher les informations des séquences
def display_sequence_info():
    # Récupérer toutes les séquences sauvegardées
    sequence_data = []

    for sars_type in types:
        for year in years:
            folder_path = os.path.join(output_dir, sars_type, str(year))
            if not os.path.exists(folder_path):
                continue

            # Charger les fichiers FASTA dans ce dossier
            fasta_files = [
                f for f in os.listdir(folder_path) if f.endswith(".fasta")
            ]

            for fasta_file in fasta_files:
                file_path = os.path.join(folder_path, fasta_file)
                record = SeqIO.read(file_path, "fasta")

                # Calculer les pourcentages de bases
                base_percentages = calculate_base_percentages(str(record.seq))

                # Ajouter les données au tableau
                sequence_data.append({
                    "ID": record.id,
                    "Année": year,
                    "Variant": sars_type,
                    "Longueur": len(record.seq),
                    "A (%)": f"{base_percentages['A']:.2f}",
                    "U (%)": f"{base_percentages['U']:.2f}",
                    "C (%)": f"{base_percentages['C']:.2f}",
                    "G (%)": f"{base_percentages['G']:.2f}",
                    "Description": record.description,
                    "Séquence": str(record.seq),
                })

    # Convertir les données en DataFrame pour un affichage propre
    df = pd.DataFrame(sequence_data)

    # Utiliser tabulate pour un affichage propre dans la console
    print(tabulate(df, headers="keys", tablefmt="grid"))

    # Retourner le DataFrame pour d'autres usages si nécessaire
    return df

# Appeler la fonction pour afficher les informations
df_sequences = display_sequence_info()


+----+----------------+---------+-----------+------------+---------+---------+---------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [27]:
# Charger un fichier FASTA dans un dossier, avec position par défaut
def get_fasta_by_position(output_dir, variant, year, position=0):
    folder_path = os.path.join(output_dir, variant, year)
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"Le dossier {folder_path} n'existe pas.")
    
    fasta_files = [f for f in os.listdir(folder_path) if f.endswith(".fasta")]
    if not fasta_files:
        raise FileNotFoundError(f"Aucun fichier FASTA trouvé dans le dossier {folder_path}.")
    
    if position < 0 or position >= len(fasta_files):
        raise IndexError(f"Position invalide : {position}. Il y a {len(fasta_files)} fichier(s) FASTA disponible(s).")
    
    # Sélectionner le fichier FASTA à la position donnée
    fasta_file = fasta_files[position]
    fasta_path = os.path.join(folder_path, fasta_file)
    print(f"Chargement du fichier FASTA : {fasta_path}")
    
    # Charger la séquence
    record = SeqIO.read(fasta_path, "fasta")
    return str(record.seq)

In [28]:
# Fonction pour l'alignement global (Needleman-Wunsch) avec affichage du score
def global_alignment(seq1, seq2, match_score=1, mismatch_penalty=-1, gap_penalty=-2):
    m = len(seq1)
    n = len(seq2)
    
    # Initialisation de la matrice de score
    score_matrix = [[0 for j in range(n+1)] for i in range(m+1)]
    
    # Initialisation des premières lignes et colonnes avec les pénalités de gap
    for i in range(1, m+1):
        score_matrix[i][0] = gap_penalty * i
    for j in range(1, n+1):
        score_matrix[0][j] = gap_penalty * j
    
    # Remplissage de la matrice de score
    for i in range(1, m+1):
        for j in range(1, n+1):
            if seq1[i-1] == seq2[j-1]:
                diag = score_matrix[i-1][j-1] + match_score
            else:
                diag = score_matrix[i-1][j-1] + mismatch_penalty
            up = score_matrix[i-1][j] + gap_penalty
            left = score_matrix[i][j-1] + gap_penalty
            score_matrix[i][j] = max(diag, up, left)
    
    # Traceback pour obtenir l'alignement
    align1 = ""
    align2 = ""
    i = m
    j = n
    alignment_score = score_matrix[i][j]  # Le score final de l'alignement global
    
    while i > 0 or j > 0:
        current_score = score_matrix[i][j]
        if i > 0 and j > 0 and ((seq1[i-1] == seq2[j-1] and current_score == score_matrix[i-1][j-1] + match_score) or
                                (seq1[i-1] != seq2[j-1] and current_score == score_matrix[i-1][j-1] + mismatch_penalty)):
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1
        elif i > 0 and current_score == score_matrix[i-1][j] + gap_penalty:
            align1 = seq1[i-1] + align1
            align2 = "-" + align2
            i -= 1
        else:
            align1 = "-" + align1
            align2 = seq2[j-1] + align2
            j -= 1
    
    return align1, align2, alignment_score

# Fonction pour l'alignement local (Smith-Waterman) avec affichage du score
def local_alignment(seq1, seq2, match_score=1, mismatch_penalty=-1, gap_penalty=-2):
    m = len(seq1)
    n = len(seq2)
    
    # Initialisation de la matrice de score
    score_matrix = [[0 for j in range(n+1)] for i in range(m+1)]
    max_score = 0
    max_pos = (0, 0)
    
    # Remplissage de la matrice de score
    for i in range(1, m+1):
        for j in range(1, n+1):
            if seq1[i-1] == seq2[j-1]:
                diag = score_matrix[i-1][j-1] + match_score
            else:
                diag = score_matrix[i-1][j-1] + mismatch_penalty
            up = score_matrix[i-1][j] + gap_penalty
            left = score_matrix[i][j-1] + gap_penalty
            score_matrix[i][j] = max(0, diag, up, left)
            if score_matrix[i][j] > max_score:
                max_score = score_matrix[i][j]
                max_pos = (i, j)
    
    # Traceback pour obtenir l'alignement
    align1 = ""
    align2 = ""
    i, j = max_pos
    alignment_score = max_score  # Le score final de l'alignement local
    
    while score_matrix[i][j] != 0:
        current_score = score_matrix[i][j]
        if i > 0 and j > 0 and ((seq1[i-1] == seq2[j-1] and current_score == score_matrix[i-1][j-1] + match_score) or
                                (seq1[i-1] != seq2[j-1] and current_score == score_matrix[i-1][j-1] + mismatch_penalty)):
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1
        elif i > 0 and current_score == score_matrix[i-1][j] + gap_penalty:
            align1 = seq1[i-1] + align1
            align2 = "-" + align2
            i -= 1
        elif j > 0 and current_score == score_matrix[i][j-1] + gap_penalty:
            align1 = "-" + align1
            align2 = seq2[j-1] + align2
            j -= 1
        else:
            break  # Si le score est 0, on arrête le traceback
    
    return align1, align2, alignment_score


# Séquences de test
seq1 = "AGTACGCA"
seq2 = "TATGC"

# Alignement global
align1_global, align2_global, score_global = global_alignment(seq1, seq2)
print("Alignement Global :")
print(align1_global)
print(align2_global)
print(f"Score d'alignement global : {score_global}")

# Alignement local
align1_local, align2_local, score_local = local_alignment(seq1, seq2)
print("\nAlignement Local :")
print(align1_local)
print(align2_local)
print(f"Score d'alignement local : {score_local}")


Alignement Global :
AGTACGCA
--TATGC-
Score d'alignement global : -3

Alignement Local :
TACGC
TATGC
Score d'alignement local : 3


In [29]:
# Récupération des séquences à partir des fichiers FASTA
seq1 = get_fasta_by_position(output_dir, 'alpha', '2020', position=0)
seq2 = get_fasta_by_position(output_dir, 'beta', '2022', position=0)

# Alignement global
align1_global, align2_global, score_global = global_alignment(seq1, seq2)
print("Alignement Global :")
print(align1_global)
print(align2_global)
print(f"Score d'alignement global : {score_global}")

# Alignement local
align1_local, align2_local, score_local = local_alignment(seq1, seq2)
print("\nAlignement Local :")
print(align1_local)
print(align2_local)
print(f"Score d'alignement local : {score_local}")

Chargement du fichier FASTA : SARS-CoV-2_Sequences\alpha\2020\MW027019.1.fasta
Chargement du fichier FASTA : SARS-CoV-2_Sequences\beta\2022\PA544048.1.fasta
Alignement Global :
------UG----G---UG-----G--C-CAA----------------AU--C-GC----AA-AC-UC---C---UGC-UG--GGA--GGUAUGA-CUUAA-UCCU-AUC---C-----G--G-----GAAC--UA-U-U----GUUUUC---------C--C---A-----UAU---A----GC-GC-CA-U-UACA-ACAG-C-CAAU-AUAG-A-UUUC--U-AUUGA-UGCUA---AUG-------G-C--U-UG---------G--------A----U-UUA-C--------CU---UU----U-----AU-AU-A-G----UUGU-A----U---------UG------C-------C-U-GG----A--AC-A---U-G--GCA-UU---C-AG---GUA----A-A-U-AUAUA-UC--CCU-AACC----G--GUCGAUG----G-G--------A-UUCUAUGC-----GGCCUA-U----U-CCACCU-G-----C-GC-CC--UAA-CGCACACU-C--GG--GA------UU-U-GA-GA--C---C---A-AU---U-U--G--ACUA-ACUG-UUCC----U-CC---U-C--------GGCA-G--CC-CGA-CGGAUA--CCUG-A---C-----U-G----A-GGA-AC------G-G-A--A--A-G--C---------U-G--C
CAAAUUUGCACUGACUUGCUUUAGCACUCAAUUUGCUUUUGCUUGUCCUGACGGCGUAAAACACGUCUAUCAGUUACGUGCCAGAUCAGUUUCACCUAAACUGUUCAUCAGACAAGAGG

In [30]:
# Séquences de test
sequences = [
    "AGTACGCA",
    "TATGC",
    "ATGACGCA"
]

# Fonction pour effectuer des alignements pair par pair
def pairwise_alignments(sequences):
    num_sequences = len(sequences)
    pairwise_results = []
    
    for i in range(num_sequences):
        for j in range(i+1, num_sequences):
            seq1 = sequences[i]
            seq2 = sequences[j]
            
            # Alignement global
            align1_global, align2_global, score_global = global_alignment(seq1, seq2)
            
            # Alignement local
            align1_local, align2_local, score_local = local_alignment(seq1, seq2)
            
            # Stocker les résultats
            result = {
                'seq1_id': f"Sequence_{i+1}",
                'seq2_id': f"Sequence_{j+1}",
                'global_alignment': (align1_global, align2_global),
                'global_score': score_global,
                'local_alignment': (align1_local, align2_local),
                'local_score': score_local
            }
            pairwise_results.append(result)
    
    return pairwise_results

In [31]:
# Fonction pour créer un alignement initial avec des gaps insérés aléatoirement
def create_initial_alignment(sequences, max_length):
    aligned_sequences = []
    for seq in sequences:
        seq = list(seq)
        # Ajouter des gaps pour atteindre la longueur maximale
        while len(seq) < max_length:
            pos = random.randint(0, len(seq))
            seq.insert(pos, '-')
        aligned_sequences.append(seq)
    return aligned_sequences

# Fonction pour calculer le score d'un alignement multiple
def alignment_score(aligned_sequences, match_score=1, mismatch_penalty=-1, gap_penalty=-2):
    score = 0
    num_sequences = len(aligned_sequences)
    length = len(aligned_sequences[0])
    for i in range(length):
        column = [seq[i] for seq in aligned_sequences]
        for j in range(num_sequences):
            for k in range(j+1, num_sequences):
                if column[j] == column[k]:
                    if column[j] != '-':
                        score += match_score
                elif column[j] == '-' or column[k] == '-':
                    score += gap_penalty
                else:
                    score += mismatch_penalty
    return score

# Fonction pour effectuer une mutation sur un alignement
def mutate_alignment(aligned_sequences):
    mutated_sequences = []
    for seq in aligned_sequences:
        seq = seq.copy()
        if random.random() < 0.1:  # Probabilité de mutation
            pos1 = random.randint(0, len(seq)-1)
            pos2 = random.randint(0, len(seq)-1)
            seq[pos1], seq[pos2] = seq[pos2], seq[pos1]
        mutated_sequences.append(seq)
    return mutated_sequences

# Fonction principale pour l'algorithme génétique
def genetic_algorithm_msa(sequences, population_size=10, generations=50):
    max_length = max(len(seq) for seq in sequences)
    # Initialisation de la population
    population = [create_initial_alignment(sequences, max_length) for _ in range(population_size)]
    
    best_alignment = None
    best_score = float('-inf')
    
    for generation in range(generations):
        # Calcul des scores pour chaque alignement
        scored_population = [(alignment, alignment_score(alignment)) for alignment in population]
        # Tri par score décroissant
        scored_population.sort(key=lambda x: x[1], reverse=True)
        # Conservation des meilleurs alignements
        population = [alignment for alignment, score in scored_population[:population_size//2]]
        
        # Mise à jour du meilleur alignement
        if scored_population[0][1] > best_score:
            best_score = scored_population[0][1]
            best_alignment = scored_population[0][0]
            print(f"Generation {generation}, Best Score: {best_score}")
        
        # Génération de nouveaux alignements par mutation
        new_population = population.copy()
        while len(new_population) < population_size:
            alignment = random.choice(population)
            mutated_alignment = mutate_alignment(alignment)
            new_population.append(mutated_alignment)
        
        population = new_population
    
    return best_alignment, best_score


In [32]:
# Fonction pour écrire les séquences dans un fichier FASTA
def write_fasta_file(sequences, file_name):
    with open(file_name, 'w') as f:
        for idx, seq in enumerate(sequences):
            f.write(f">Sequence_{idx+1}\n")
            f.write(seq + "\n")

import subprocess

# Fonction pour exécuter ClustalW
def run_clustalw(fasta_file):
    clustalw_exe = 'clustalw2'  # Assurez-vous que le nom correspond à votre installation
    try:
        subprocess.run([clustalw_exe, '-INFILE=' + fasta_file], check=True)
    except subprocess.CalledProcessError as e:
        print("Erreur lors de l'exécution de ClustalW :", e)

# Fonction pour lire le fichier d'alignement ClustalW
def read_clustalw_alignment(aln_file):
    alignment = AlignIO.read(aln_file, 'clustal')
    return alignment

# Calcul du score de l'alignement ClustalW
def alignment_score_from_alignment(alignment, match_score=1, mismatch_penalty=-1, gap_penalty=-2):
    aligned_sequences = [list(record.seq) for record in alignment]
    return alignment_score(aligned_sequences, match_score, mismatch_penalty, gap_penalty)



In [33]:
# Exécuter les alignements pair par pair
pairwise_results = pairwise_alignments(sequences)

# Affichage des résultats des alignements pair par pair
print("=== Alignements pair par pair ===\n")
for result in pairwise_results:
    print(f"Alignement entre {result['seq1_id']} et {result['seq2_id']}:")
    print("Alignement Global :")
    print(result['global_alignment'][0])
    print(result['global_alignment'][1])
    print(f"Score d'alignement global : {result['global_score']}\n")
    print("Alignement Local :")
    print(result['local_alignment'][0])
    print(result['local_alignment'][1])
    print(f"Score d'alignement local : {result['local_score']}")
    print("-" * 50)

# Étape 2 : Algorithme génétique pour l'alignement multiple
best_alignment, best_score = genetic_algorithm_msa(sequences, population_size=10, generations=50)

# Convertir l'alignement en une liste de chaînes de caractères
aligned_sequences_ga = [''.join(seq) for seq in best_alignment]

# Affichage de l'alignement multiple obtenu
print("\n=== Alignement multiple obtenu par l'algorithme génétique ===\n")
for idx, seq in enumerate(aligned_sequences_ga):
    print(f"Sequence_{idx+1}:\t{seq}")

print(f"\nScore de l'alignement génétique : {best_score}")

# Étape 3 : Alignement multiple avec ClustalW
fasta_file = 'sequences.fasta'
write_fasta_file(sequences, fasta_file)
run_clustalw(fasta_file)
aln_file = 'sequences.aln'
alignment_clustalw = read_clustalw_alignment(aln_file)

# Affichage de l'alignement
print("\n=== Alignement multiple avec ClustalW ===\n")
for record in alignment_clustalw:
    print(f"{record.id}:\t{record.seq}")
score_clustalw = alignment_score_from_alignment(alignment_clustalw)
print(f"\nScore de l'alignement ClustalW : {score_clustalw}")

# Étape 4 : Comparaison des résultats
print("\n=== Comparaison des scores d'alignement ===\n")
print(f"Score de l'alignement génétique : {best_score}")
print(f"Score de l'alignement ClustalW : {score_clustalw}")

# Affichage des alignements multiples pour comparaison
print("\n=== Alignement multiple obtenu par l'algorithme génétique ===\n")
for idx, seq in enumerate(aligned_sequences_ga):
    print(f"Sequence_{idx+1}:\t{seq}")

print("\n=== Alignement multiple obtenu par ClustalW ===\n")
for record in alignment_clustalw:
    print(f"{record.id}:\t{record.seq}")


=== Alignements pair par pair ===

Alignement entre Sequence_1 et Sequence_2:
Alignement Global :
AGTACGCA
--TATGC-
Score d'alignement global : -3

Alignement Local :
TACGC
TATGC
Score d'alignement local : 3
--------------------------------------------------
Alignement entre Sequence_1 et Sequence_3:
Alignement Global :
AGTACGCA
ATGACGCA
Score d'alignement global : 4

Alignement Local :
ACGCA
ACGCA
Score d'alignement local : 5
--------------------------------------------------
Alignement entre Sequence_2 et Sequence_3:
Alignement Global :
-T-ATGC-
ATGACGCA
Score d'alignement global : -3

Alignement Local :
ATG
ATG
Score d'alignement local : 3
--------------------------------------------------
Generation 0, Best Score: -8
Generation 27, Best Score: -4
Generation 32, Best Score: 2

=== Alignement multiple obtenu par l'algorithme génétique ===

Sequence_1:	ATGACGCA
Sequence_2:	AT-T-GC-
Sequence_3:	ATGACGCA

Score de l'alignement génétique : 2

=== Alignement multiple avec ClustalW ===

Se